In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB2
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.options.display.max_colwidth=999
train = pd.read_csv('/kaggle/input/crop-pest-classification/train.csv')
train['path'] = '../input/crop-pest-classification/'+train['img_path']
train['disease_code'] = train['disease_code'].astype(str)
train

In [ ]:
test = pd.read_csv('/kaggle/input/crop-pest-classification/test.csv')
test['path'] = '../input/crop-pest-classification/'+test['img_path']
test

In [ ]:
# KFold(enumerate 관련 확인차원)

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator

skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
idg = ImageDataGenerator(horizontal_flip=True)
idg_2 = ImageDataGenerator(horizontal_flip=True)
for i,(train_index,valid_index) in enumerate(skf.split(train,train['disease_code'])):
    X_train = train.iloc[train_index]
    X_valid = train.iloc[valid_index]
    train_generator = idg.flow_from_dataframe(X_train,x_col='path',y_col='disease_code',
                                             target_size=(150,150),batch_size=32)
    valid_generator = idg_2.flow_from_dataframe(X_valid,x_col='path',y_col='disease_code',
                                             target_size=(150,150),batch_size=32)
    print(X_train)
    print(len(X_train))

In [ ]:
idg_3 = ImageDataGenerator()
test_generator = idg_3.flow_from_dataframe(test,x_col='path',y_col=None,
                                        class_mode=None,shuffle=None,target_size=(400,400),
                                        batch_size=16)

In [ ]:
# horizontal_flip 관련 확인
# rotation_range 관련 확인
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import matplotlib.pyplot as plt
image = load_img('../input/crop-pest-classification/train_imgs/10036.jpg')
image

In [ ]:
data = img_to_array(image)
sample = np.expand_dims(data,0)
# np.expand_dims : augmentation을 위해 data를 새로 넣어주고 싶은데 data의 차원은 3차원
#             --> 0 은 axis=0
sample.shape
# 맨 앞에 batch관련 차원 추가(이미지 1장)

In [ ]:
sample_gen = ImageDataGenerator(rotation_range=(30))  # rotation_range : 회전 각도 설정
sample_generator = sample_gen.flow(sample,batch_size=1)
plt.figure(figsize=(15,15))
for i in range(9):
    plt.subplot(3,3,i+1)
    batch = sample_generator.next()  # sample_generator에 들어있는 데이터 뽑아옴
    image = batch[0].astype('uint8') # 그림 잘 보이게 픽셀값 조정
    plt.imshow(image)

### horizontal_flip & rotation_range 관련 제출시도해보기

In [ ]:
# 1) horizontal_flip

result = 0
skf = StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
idg = ImageDataGenerator(horizontal_flip=True)
idg_2 = ImageDataGenerator()
for i, (train_index,valid_index) in enumerate(skf.split(train,train['disease_code'])) :
    X_train = train.iloc[train_index]
    X_valid = train.iloc[valid_index]
    train_generator = idg.flow_from_dataframe(X_train,x_col='path',y_col='disease_code',
                                             target_size=(400,400),batch_size=16)
    valid_generator = idg_2.flow_from_dataframe(X_valid,x_col='path',y_col='disease_code',
                                               target_size=(400,400),batch_size=16)
    es = EarlyStopping(patience=5,restore_best_weights=True,verbose=1)
    rl = ReduceLROnPlateau(patience=4)
    efn = EfficientNetB2(include_top=False,pooling='avg')
    # efn을 B1, B2, B3 등등으로 실험해보기!
    # 너무 복잡한 모델로 시도시 RAM 터질 가능성 있음
    # --> batch_size를 줄여줘야함
    model = Sequential()
    model.add(efn)
    model.add(Dense(7,activation='softmax'))
    model.compile(metrics=['acc'],loss='categorical_crossentropy',optimizer='adam')
    model.fit(train_generator,validation_data=valid_generator,epochs=100,callbacks=[es,rl])
    result += model.predict(test_generator) / 10

In [ ]:
# 2) rotation_range적용 --> horizontal_flip을 그냥 rotation_range적어주면 됨!
result = 0
skf = StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
idg = ImageDataGenerator(rotation_range=(30))
idg_2 = ImageDataGenerator()  # 평가시에는 horizontal_flip 필요 x
for i,(train_index,valid_index) in enumerate(skf.split(train,train['disease_code'])):
    X_train = train.iloc[train_index]
    X_valid = train.iloc[valid_index]
    train_generator = idg.flow_from_dataframe(X_train,x_col='path',y_col='disease_code',
                                             target_size=(400,400),batch_size=16)
    valid_generator = idg_2.flow_from_dataframe(X_valid,x_col='path',y_col='disease_code',
                                             target_size=(400,400),batch_size=16)
    es = EarlyStopping(patience=5,restore_best_weights=True,verbose=1)
    rl = ReduceLROnPlateau(patience=4)
    efn = EfficientNetB2(include_top=False,pooling='avg')
    # efn을 B1, B2, B3 등등으로 실험해보기!
    # 너무 복잡한 모델로 시도시 RAM 터질 가능성 있음
    # --> batch_size를 줄여줘야함
    model = Sequential()
    model.add(efn)
    model.add(Dense(7,activation='softmax'))
    model.compile(metrics=['acc'],loss='categorical_crossentropy',optimizer='adam')
    model.fit(train_generator,epochs=100,validation_data=valid_generator,callbacks=[es,rl])
    result += model.predict(test_generator) / 10

In [ ]:
result

In [ ]:
result.argmax(1)

In [ ]:
class_index = list(train_generator.class_indices)
class_index

In [ ]:
sub = pd.read_csv('/kaggle/input/crop-pest-classification/sample_submission.csv')
sub['disease_code'] = result.argmax(1)
sub.to_csv('result.csv',index=False)